In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
spark = (SparkSession
  .builder
  .appName("US_Accidents")
  .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/03 00:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([StructField('ID', StringType(), False),
                StructField('Source', StringType(), False),
                StructField('Severity', IntegerType(), False),
                StructField('Start_Time', TimestampType(), False),                  
                StructField('End_Time', TimestampType(), False),      
                StructField('Start_Lat', DecimalType(), False),
                StructField('Start_Lng', DecimalType(), False),
                StructField('End_Lat', DecimalType(), True),
                StructField('End_Lng', DecimalType(), True),       
                StructField('Distance(mi)', DecimalType(), False),       
                StructField('Description', StringType(), False),       
                StructField('Street', StringType(), True),                 
                StructField('City', StringType(), True),       
                StructField('County', StringType(), True),       
                StructField('State', StringType(), True),       
                StructField('Zipcode', StringType(), True),       
                StructField('Country', StringType(), True),       
                StructField('Timezone', StringType(), True),       
                StructField('Airport_Code', StringType(), True),
                StructField('Weather_Timestamp', TimestampType(), True),
                StructField('Temperature(F)', DecimalType(), True),
                StructField('Wind_Chill(F)', DecimalType(), True),
                StructField('Humidity(%)', DecimalType(), True),
                StructField('Pressure(in)', DecimalType(), True),
                StructField('Visibility(mi)', DecimalType(), True),
                StructField('Wind_Direction', StringType(), True),
                StructField('Wind_Speed(mph)', DecimalType(), True),
                StructField('Precipitation(in)', DecimalType(), True),
                StructField('Weather_Condition', StringType(), True),
                StructField('Amenity', BooleanType(), False),
                StructField('Bump', BooleanType(), False),
                StructField('Crossing', BooleanType(), False),
                StructField('Give_way', BooleanType(), False),
                StructField('Junction', BooleanType(), False),
                StructField('No_Exit', BooleanType(), False),
                StructField('Railway', BooleanType(), False), 
                StructField('Roundabout', BooleanType(), False),
                StructField('Station', BooleanType(), False),
                StructField('Stop', BooleanType(), False),
                StructField('Traffic_Calming', BooleanType(), False),
                StructField('Traffic_Signal', BooleanType(), False),
                StructField('Turning_Loop', BooleanType(), False),
                StructField('Sunrise_Sunset', StringType(), True),
                StructField('Civil_Twilight', StringType(), True),
                StructField('Nautical_Twilight', StringType(), True),
                StructField('Astronomical_Twilight', StringType(), True)])

# Use the DataFrameReader interface to read a CSV file
file = "US_Accidents_March23.csv"
df = spark.read.csv(file, header=True, schema=schema)

In [5]:
df = df.select('ID', 'Start_Time', 'End_Time', 'Timezone', 'Weather_Timestamp', 'Weather_Condition')
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|
+---+-------------------+-------------------+----------+-------------------+-----------------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|
|A-3|2016-02-08 06:49:27|2016-02-08 07:19:27|US/Eastern|2016-02-08 06:56:00|         Overcast|
|A-4|2016-02-08 07:23:34|2016-02-08 07:53:34|US/Eastern|2016-02-08 07:38:00|    Mostly Cloudy|
|A-5|2016-02-08 07:39:07|2016-02-08 08:09:07|US/Eastern|2016-02-08 07:53:00|    Mostly Cloudy|
+---+-------------------+-------------------+----------+-------------------+-----------------+
only showing top 5 rows



In [6]:
# Get row count
rows = df.count()
print(f"DataFrame Rows count : {rows}")

# Get columns count
cols = len(df.columns)
print(f"DataFrame Columns count : {cols}")

DataFrame Rows count : 7728394
DataFrame Columns count : 6


In [7]:
df.groupBy('Timezone').count().orderBy(desc("count")).show()

+-----------+-------+
|   Timezone|  count|
+-----------+-------+
| US/Eastern|3580167|
| US/Pacific|2062984|
| US/Central|1645616|
|US/Mountain| 431819|
|       null|   7808|
+-----------+-------+



In [8]:
#have to remove the ~8000 rows with null timezone if using date/time in any way
df = df.dropna(subset=["Timezone"])
df.groupBy('Timezone').count().orderBy(desc("count")).show()

+-----------+-------+
|   Timezone|  count|
+-----------+-------+
| US/Eastern|3580167|
| US/Pacific|2062984|
| US/Central|1645616|
|US/Mountain| 431819|
+-----------+-------+



In [10]:
#Null counts (excluding time columns)
df.agg(*[
    (df.count()-count(c)).alias(c)    # vertical (column-wise) operations in SQL ignore NULLs
    for c in df.columns 
]).show()

+---+----------+--------+--------+-----------------+-----------------+
| ID|Start_Time|End_Time|Timezone|Weather_Timestamp|Weather_Condition|
+---+----------+--------+--------+-----------------+-----------------+
|  0|         0|       0|       0|           112420|           165651|
+---+----------+--------+--------+-----------------+-----------------+



In [11]:
# Convert times to utc in new columns UTC_*
df=df.withColumn("Start_Time",col("Start_Time").cast("Timestamp"))\
.withColumn("UTC_Start_Time",to_utc_timestamp(col("Start_Time"), col("Timezone")))\
.withColumn("End_Time",col("End_Time").cast("Timestamp"))\
.withColumn("UTC_End_Time",to_utc_timestamp(col("End_Time"), col("Timezone")))\
.withColumn("Weather_Timestamp",col("Weather_Timestamp").cast("Timestamp"))\
.withColumn("UTC_Weather_Timestamp",to_utc_timestamp(col("Weather_Timestamp"), col("Timezone")))
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|2016-02-08 11:07:59|2016-02-08 11:37:59|  2016-02-08 10:51:00|
|A-3|2016-02-08 06:49:27|2016-02-08 07:19:27|US/Eastern|2016-02-08 06:56:00|         Overcast|2016-02-08 11:49:27|2016-02-08 12:19:27|  2016-02-08 11:56:00|
|A-4|2016-02-08 07:23:34|2016-02-08 07:53:34|US/Eastern|20

In [12]:
# Create Day of the Week Column
df = df.withColumn('Day_of_Week', dayofweek(col('Start_Time'))) #Ranges from 1 for a Sunday through to 7 for a Saturday
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|Day_of_Week|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|          2|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|2016-02-08 11:07:59|2016-02-08 11:37:59|  2016-02-08 10:51:00|          2|
|A-3|2016-02-08 06:49:27|2016-02-08 07:19:27|US/Eastern|2016-02-08 06:56:00|         Overcast|2016-02-08 11:49:27|2016-02-08 12:19:27|  2016-02-08 11:56:00

In [13]:
# Create Month Column
df = df.withColumn('Month', month(df['Start_Time']))
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|Day_of_Week|Month|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|          2|    2|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|2016-02-08 11:07:59|2016-02-08 11:37:59|  2016-02-08 10:51:00|          2|    2|
|A-3|2016-02-08 06:49:27|2016-02-08 07:19:27|US/Eastern|2016-02-08 06:56:00|         Overcast|2016-02-08 11:49:27|2016-02-08 

In [14]:
# Create Season Column
import pyspark.sql.functions as F
df = df.withColumn(
    'Season',
    F.when(F.col("Month").between(3, 5), 'Spring')\
    .when(F.col("Month").between(6, 8), 'Summer')\
    .when(F.col("Month").between(9, 11), 'Fall')\
    .otherwise('Winter'))
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|Day_of_Week|Month|Season|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|          2|    2|Winter|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|2016-02-08 11:07:59|2016-02-08 11:37:59|  2016-02-08 10:51:00|          2|    2|Winter|
|A-3|2016-02-08 06:49:27|2016-02-08 07:19:27|US/Eastern|2016-02-08 06:56:00|         Overc

In [15]:
# Create Rush Hour Column
# defined as 6-9am and 4-7pm (https://www.goldlaw.com/how-deadly-is-rush-hour/)
df = df.withColumn('utc_start_time_Time', date_format('UTC_Start_Time', 'HH:mm:ss'))
df.show(5)
df = df.withColumn(
    'Rush Hour',
    F.when(F.col('utc_start_time_Time').between('06:00:00', '09:00:00'), 1)\
    .when(F.col('utc_start_time_Time').between('16:00:00', '19:00:00'), 1)\
    .otherwise(0))
df.show(5)

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+-------------------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|Day_of_Week|Month|Season|utc_start_time_Time|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+-------------------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|          2|    2|Winter|           10:46:00|
|A-2|2016-02-08 06:07:59|2016-02-08 06:37:59|US/Eastern|2016-02-08 05:51:00|       Light Rain|2016-02-08 11:07:59|2016-02-08 11:37:59|  2016-02-08 10:51:00|          2|    2|Winter|           

In [16]:
# Create Date Column from UTC Start Date
df = df.withColumn('utc_start_time_Date', date_format('UTC_Start_Time', 'yyyy-MM-dd'))

#Create list of holiday dates for span of data (2016-2023)
#!pip3 install holidays
import holidays
from datetime import datetime, timezone
h = []
for ptr in holidays.UnitedStates(years = [2016,2017,2018,2019,2020,2021,2022,2023]).items(): 
    h.append(ptr[0].strftime("%Y-%m-%d"))

# Create Holiday column    
df = df.withColumn(
    'Holiday',
    F.when(array_contains(lit(h), F.col("utc_start_time_Date")), 1)\
    .otherwise(0))
df.show(5) 

+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+-------------------+---------+-------------------+-------+
| ID|         Start_Time|           End_Time|  Timezone|  Weather_Timestamp|Weather_Condition|     UTC_Start_Time|       UTC_End_Time|UTC_Weather_Timestamp|Day_of_Week|Month|Season|utc_start_time_Time|Rush Hour|utc_start_time_Date|Holiday|
+---+-------------------+-------------------+----------+-------------------+-----------------+-------------------+-------------------+---------------------+-----------+-----+------+-------------------+---------+-------------------+-------+
|A-1|2016-02-08 05:46:00|2016-02-08 11:00:00|US/Eastern|2016-02-08 05:58:00|       Light Rain|2016-02-08 10:46:00|2016-02-08 16:00:00|  2016-02-08 10:58:00|          2|    2|Winter|           10:46:00|        0|         2016-02-08|      0|
|A-2|2016-02-08 06:07:59|2016-02-08 06:3

In [ ]:
# Weather Condition

In [18]:
df.select('Weather_Condition').distinct().count()

145

In [20]:
df.select('Weather_Condition').distinct().show(150, truncate = False)
#Could create Binary columns for a few features where words like Fog, Snow, etc. are included

+-----------------------------------+
|Weather_Condition                  |
+-----------------------------------+
|Ice Pellets                        |
|Shallow Fog                        |
|Thunderstorm                       |
|Volcanic Ash                       |
|N/A Precipitation                  |
|Showers in the Vicinity            |
|Cloudy                             |
|Light Freezing Rain                |
|Blowing Snow                       |
|Heavy Rain / Windy                 |
|Low Drifting Snow                  |
|Heavy Thunderstorms and Snow       |
|null                               |
|Widespread Dust                    |
|Snow Grains                        |
|Light Rain with Thunder            |
|Squalls                            |
|Scattered Clouds                   |
|Heavy T-Storm                      |
|Patches of Fog                     |
|Rain Showers                       |
|Thunderstorms and Rain             |
|Drizzle                            |
|Cloudy / Wi

In [14]:
df.groupBy('Weather_Condition').count().orderBy(desc("count")).show()
#Fog/Haze, Snow

+--------------------+-------+
|   Weather_Condition|  count|
+--------------------+-------+
|                Fair|2560802|
|       Mostly Cloudy|1016195|
|              Cloudy| 817082|
|               Clear| 808743|
|       Partly Cloudy| 698972|
|            Overcast| 382866|
|          Light Rain| 352957|
|    Scattered Clouds| 204829|
|                null| 173459|
|          Light Snow| 128680|
|                 Fog|  99238|
|                Rain|  84331|
|                Haze|  76223|
|        Fair / Windy|  35671|
|          Heavy Rain|  32309|
|       Light Drizzle|  22684|
|Thunder in the Vi...|  17611|
|      Cloudy / Windy|  17035|
|             T-Storm|  16810|
|Mostly Cloudy / W...|  16508|
+--------------------+-------+
only showing top 20 rows

